# 2019-02-03_Campbell_26

| Info   |                      |
|--------|----------------------|
| Author | L. D. Nicolas May    |
| Date   | 2019-02-23           |
| Note   | This is a re-creation of a dataset that was built using R in order to get data munging experience with Python |

## Load Requirements

### Load useful globals and functions

In [1]:
from config import *

### Load packages

In [2]:
# Package(s) for REDCap API
import pycurl
from io import BytesIO
import certifi

# Package(s) for munging data
import pandas as pd
import numpy as np

## Get Data

### Define data fields and forms

#### UDS 3

In [3]:
#---------#---------#---------#---------#---------#---------#---------#---------

# FIELDS

# Form Header
fields_u3_hd_raw = [
    'ptid'
    , 'form_date'
]
# Form C2 - IVP
fields_u3_c2_raw = [
    'mocatots'
    , 'trailb_c2'
    , 'trailbrr_c2'
    , 'trailbli_c2'
    , 'trailb_c2z'
]
# Form D1 - IVP
fields_u3_d1_raw = [
    'normcog'    # NL
    , 'mciamem'  # MCI
    , 'mciaplus' # MCI
    , 'mcinon1'  # MCI
    , 'mcinon2'  # MCI
    , 'impnomci' # Impaired not MCI
    , 'alzdis'   # AD etio
    , 'alzdisif'
    , 'lbdis'    # LBD etio
    , 'lbdif'
    , 'msa'      # MSA etio
    , 'msaif'
    , 'psp'      # PSP etio
    , 'pspif'
    , 'ftldmo'   # FTLD motor etio
    , 'ftldmoif'
    , 'ftldnos'  # FTLD NOS etio
    , 'ftldnoif'
]
# Combine fields, prefixing C2 and D1 with "fu_" and "tele_"
fields_u3_raw = \
    fields_u3_hd_raw + \
    fields_u3_c2_raw + \
    ['fu_' + f for f in fields_u3_c2_raw] + \
    ['tele_' + f for f in fields_u3_c2_raw] + \
    fields_u3_d1_raw + \
    ['fu_' + f for f in fields_u3_d1_raw] + \
    ['tele_' + f for f in fields_u3_d1_raw]

fields_u3 = ','.join(fields_u3_raw)
# print(fields_u3)


# FORMS

forms_u3_raw = [
    # Lichtenberg Financial Decision Making Self Efficacy Form
    'financial_decision_making_self_efficacy_form'
    # Lichtenberg Financial Decision Making Screening Scale
    , 'lfdss_for_professionals'
]

forms_u3 = ','.join(forms_u3_raw)
# print(forms_u3)

In [5]:
#---------#---------#---------#---------#---------#---------#---------#---------

# certifi.where()

data = [
    ('token',                  ('6113F92032EA4C054E574EBD226AFBAF')),
    ('content',                ('record')),
    ('format',                 ('json')),
    ('type',                   ('flat')),
    ('rawOrLabel',             ('raw')),
    ('rawOrLabelHeaders',      ('raw')),
    ('exportCheckboxLabel',    ('false')),
    ('exportSurveyFields',     ('false')),
    ('exportDataAccessGroups', ('false')),
    ('returnFormat',           ('json')),
    ('fields',                 (fields_u3)),
    ('forms',                  (forms_u3))
]

buf = BytesIO()

c = pycurl.Curl()

c.setopt(pycurl.CAINFO, certifi.where())
c.setopt(pycurl.URL, REDCAP_API_URI)
c.setopt(pycurl.HTTPPOST, data)
c.setopt(c.WRITEFUNCTION, buf.write)
c.setopt(c.SSL_VERIFYPEER, False)
# ^^^ UM MICHR REDCap server can't verify local certificate (???)

c.perform()
c.close()

In [6]:
# Get JSON
json_u3 = buf.getvalue()

In [7]:
# Parse JSON as pandas df
df_u3 = pd.read_json(json_u3, convert_dates=['form_date'])

In [8]:
# Write to CSV
df_u3.to_csv('py_df_u3.csv')

# Read CSV
df_u3 = pd.read_csv('py_df_u3.csv', sep=',')

In [9]:
# df_u3.head(n=10)

In [10]:
df_u3.dtypes

Unnamed: 0                                                 int64
age_categories_4                                         float64
alzdis                                                   float64
alzdisif                                                 float64
case_outcom___1                                          float64
case_outcom___2                                          float64
details_change_plans                                      object
details_financial_decision                                object
details_financial_goal                                    object
details_impact                                            object
details_neg_affected                                      object
details_of_purpose                                        object
details_of_whose_idea                                     object
details_risk                                              object
details_talk_to_anyone                                    object
details_who_benefits     

In [11]:
# Reorder `df_u3` columns as they're originally listed

# fields_u3_raw[1:20]

orig_cols = df_u3.columns.tolist()
# cols

ordered_cols = list(filter(lambda f: f in orig_cols, fields_u3_raw))
# ordered_cols
remaining_cols = list(filter(lambda f: f not in ordered_cols, orig_cols))
# remaining_cols
reordered_cols = ordered_cols + remaining_cols

df_u3 = df_u3[reordered_cols]

## Process Data

### Clean Data

#### UDS 3

Clean out DDE records (`--1`, `--2`)

In [12]:
# df_u3['ptid'].str.match(r'^UM\d{8}$')
df_u3_cln = df_u3.loc[df_u3['ptid'].str.match(r'^UM\d{8}$')]

Clean out records missing `form_date`s

In [13]:
# df_u3['form_date'].notnull()
df_u3_cln = df_u3.loc[df_u3['form_date'].notnull()]

### Mutate Data

#### UDS 3